In [6]:
import pandas as pd
import tensorflow as tf

# Reading Data

In [9]:
#Reading the dataset
data=pd.read_csv("news.csv")
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
data.shape

(20800, 5)

In [11]:
data['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [12]:
#Creating the content column which contains the author name and title of the news
data['content']=data['author'] + ' ' + data['title']
data['content'][4]

'Howard Portnoy Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery'

In [13]:
#Checking how many NULL values are there
data.isna().sum()

id            0
title       558
author     1957
text         39
label         0
content    2515
dtype: int64

In [14]:
#Dropping all the Null Values
data=data.dropna() #Drop NAN value
data.isna().sum()

id         0
title      0
author     0
text       0
label      0
content    0
dtype: int64

# Preprocessing the data

In [22]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re

#Converting text into data required for training
def convert(text):
    ps=PorterStemmer()
    text_converted=re.sub('[^a-zA-Z]',' ',text)
    text_converted=text_converted.lower()
    text_converted=text_converted.split()
    text_stemmed=[ps.stem(word) for word in text_converted if word not in stopwords.words('english')]
    text_stemmed=' '.join(text_stemmed)
    return text_stemmed

In [23]:
#Applying the convert function in each text in content column
data['content']=data['content'].apply(convert)
data['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 18285, dtype: object

In [24]:
X=data['content'].values
y=data['label'].values

In [25]:
X,y

(array(['darrel lucu hous dem aid even see comey letter jason chaffetz tweet',
        'daniel j flynn flynn hillari clinton big woman campu breitbart',
        'consortiumnew com truth might get fire', ...,
        'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time',
        'alex ansari nato russia hold parallel exercis balkan',
        'david swanson keep f aliv'], dtype=object),
 array([1, 0, 1, ..., 0, 1, 1], dtype=int64))

In [26]:
#Converting text data into numbers
from sklearn.feature_extraction.text import TfidfVectorizer
vector=TfidfVectorizer()
vector.fit(X)
X=vector.transform(X)

In [27]:
print(X[0])

  (0, 14626)	0.2853880981846006
  (0, 12567)	0.25566372256502734
  (0, 8310)	0.3609049070394367
  (0, 8048)	0.29347549279156676
  (0, 7190)	0.24556189342497173
  (0, 6552)	0.21745594418933306
  (0, 4637)	0.23016077319140021
  (0, 3543)	0.2684494960336511
  (0, 3359)	0.3609049070394367
  (0, 2757)	0.2466340295002162
  (0, 2312)	0.3745612250433202
  (0, 247)	0.26982554594264346


# Spliting the dataset

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
X_train.shape,y_train.shape

((14628, 15960), (14628,))

# Model Training

In [33]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(100,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss="binary_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=10)

Epoch 1/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8167 - loss: 0.4438 - val_accuracy: 0.9880 - val_loss: 0.0665
Epoch 2/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9965 - loss: 0.0312 - val_accuracy: 0.9915 - val_loss: 0.0376
Epoch 3/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9991 - loss: 0.0091 - val_accuracy: 0.9926 - val_loss: 0.0304
Epoch 4/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 0.0037 - val_accuracy: 0.9929 - val_loss: 0.0274
Epoch 5/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 0.9932 - val_loss: 0.0256
Epoch 6/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.9926 - val_loss: 0.0247
Epoch 7/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 8.2159e-04 - val_accuracy: 0.9929 - val_loss: 0.0241
Epoch 8/10
458/458 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 5.3761e-04 - val_accu

In [34]:
#Evaluating model performance
model.evaluate(X_test,y_test)

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 932us/step - accuracy: 0.9926 - loss: 0.0257


[0.023345068097114563, 0.9934372305870056]

In [35]:
y_pred=model.predict(X_test)
y_pred

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


array([[9.9998415e-01],
       [9.6794300e-02],
       [4.6051590e-04],
       ...,
       [9.9999541e-01],
       [1.1964350e-03],
       [9.9982506e-01]], dtype=float32)

In [36]:
data['content'][0]



'darrel lucu hous dem aid even see comey letter jason chaffetz tweet'

In [37]:
ans=model.predict(X_test[0])
if(ans[0]>=0.5):
    print("real")
else:
    print("False")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
real


In [38]:
X_test[5]

<1x15960 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [39]:
y_test[0]

1

In [78]:
news={
    1 :"Real",
    0:"Fake"
}
def result(X,y,index):
    ans=model.predict(X[index])
    print("Tensorflow model result : ")
    if(ans[0]>=0.5):
        print("Predicted value:Real")
        print(f"True value:{news[y[index]]}")
    else:
        print("Predicted value:Fake")
        print(f"True value:{news[y[index]]}")

In [86]:
result(X_test,y_test,900)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Tensorflow model result : 
Predicted value:Fake
True value:Fake
